<a href="https://colab.research.google.com/github/Vishal8500/Text-to-Speech-DL-Project/blob/main/Text_to_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa numpy pandas matplotlib tqdm

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/LJSpeech-1.1"

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

# Set dataset paths
DATASET_PATH = "/content/drive/MyDrive/LJSpeech-1.1"  # Update this if needed
AUDIO_PATH = os.path.join(DATASET_PATH, "wavs")
METADATA_FILE = os.path.join(DATASET_PATH, "metadata.csv")
OUTPUT_PATH = "/content/processed_data"

# Create output directory
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Load metadata
df = pd.read_csv(METADATA_FILE, sep="|", header=None, names=["file_id", "transcript", "normalized_text"])

# Handle missing or non-string values
df["normalized_text"] = df["normalized_text"].fillna("").astype(str)

# Function to clean text
def clean_text(text):
    text = text.lower().strip()
    text = text.replace(",", "").replace(".", "").replace(";", "").replace("?", "")
    return text

# Apply text cleaning
df["clean_text"] = df["normalized_text"].apply(clean_text)

# Function to convert audio to Mel-spectrogram
def process_audio(file_id, sr=22050, n_mels=80):
    file_path = os.path.join(AUDIO_PATH, f"{file_id}.wav")
    y, _ = librosa.load(file_path, sr=sr)

    # Convert to Mel-spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    return mel_spec_db

# Process all audio files and save
mel_data = {}

for idx, row in tqdm(df.iterrows(), total=len(df)):
    file_id = row["file_id"]
    mel_spec_db = process_audio(file_id)

    # Save spectrogram as numpy file
    np.save(os.path.join(OUTPUT_PATH, f"{file_id}.npy"), mel_spec_db)

    # Store metadata (text and spectrogram shape)
    mel_data[file_id] = {"text": row["clean_text"], "mel_shape": mel_spec_db.shape}

# Save metadata to JSON
with open(os.path.join(OUTPUT_PATH, "metadata.json"), "w") as f:
    json.dump(mel_data, f, indent=4)

print("✅ Preprocessing complete! Mel-spectrograms and metadata saved.")


100%|██████████| 13100/13100 [2:53:43<00:00,  1.26it/s]

✅ Preprocessing complete! Mel-spectrograms and metadata saved.


In [ ]:
import shutil

# Zip the processed_data folder
shutil.make_archive('/content/processed_data', 'zip', '/content/processed_data')

# Download the zip file
from google.colab import files
files.download('/content/processed_data.zip')


FileNotFoundError: [Errno 2] No such file or directory: '/content/processed_data'

In [ ]:
# Install required libraries
!pip install tensorflow librosa matplotlib


In [ ]:
# Example: TensorFlow Dataset loading (you can use PyTorch similarly)
import tensorflow as tf

def load_data(metadata_path, audio_path, batch_size=32):
    with open(metadata_path, "r") as f:
        metadata = json.load(f)

    text_data = [entry["text"] for entry in metadata.values()]
    mel_data = [np.load(os.path.join(audio_path, f"{file_id}.npy")) for file_id in metadata.keys()]

    # Convert to TensorFlow Dataset (you can also use PyTorch Dataset)
    dataset = tf.data.Dataset.from_tensor_slices((text_data, mel_data))
    dataset = dataset.batch(batch_size)
    return dataset


In [ ]:
pip install torch torchvision numpy tensorboard scipy matplotlib librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
!python --version


Python 3.11.11


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.7 python3.7-distutils -y


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,698 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [47.7 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,238 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,813 kB]
Get:13 https:/

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3


There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.11   2         auto mode
  1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.11   2         manual mode
  3            /usr/bin/python3.7    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 3
update-alternatives: using /usr/bin/python3.7 to provide /usr/bin/python3 (python3) in manual mode


In [ ]:
!python --version


Python 3.7.17


In [1]:
!git clone https://github.com/NVIDIA/tacotron2.git

Cloning into 'tacotron2'...
remote: Enumerating objects: 412, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 412 (delta 2), reused 1 (delta 1), pack-reused 408 (from 2)
Receiving objects: 100% (412/412), 2.70 MiB | 6.09 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [2]:
%cd tacotron2

/content/tacotron2


In [3]:
!cat /content/tacotron2/model.py


from math import sqrt
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F
from layers import ConvNorm, LinearNorm
from utils import to_gpu, get_mask_from_lengths


class LocationLayer(nn.Module):
    def __init__(self, attention_n_filters, attention_kernel_size,
                 attention_dim):
        super(LocationLayer, self).__init__()
        padding = int((attention_kernel_size - 1) / 2)
        self.location_conv = ConvNorm(2, attention_n_filters,
                                      kernel_size=attention_kernel_size,
                                      padding=padding, bias=False, stride=1,
                                      dilation=1)
        self.location_dense = LinearNorm(attention_n_filters, attention_dim,
                                         bias=False, w_init_gain='tanh')

    def forward(self, attention_weights_cat):
        processed_attention = self.location_conv(attention_weights_cat)
        processed

In [6]:
import torch
from model import Tacotron2
from hparams import create_hparams

hparams = create_hparams()
model = Tacotron2(hparams)

# Print model structure
print(model)


AttributeError: module 'tensorflow' has no attribute 'contrib'

In [ ]:
!wget https://bootstrap.pypa.io/pip/3.7/get-pip.py
!python3.7 get-pip.py


--2025-03-25 15:53:56--  https://bootstrap.pypa.io/pip/3.7/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2636033 (2.5M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.51M  --.-KB/s    in 0.03s   

2025-03-25 15:53:56 (85.3 MB/s) - ‘get-pip.py’ saved [2636033/2636033]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.3 MB/s eta 0:00:00


In [5]:
!pip install numpy==1.18.5
!pip install matplotlib==3.2.2
!pip install tensorflow==1.15.2
!pip install inflect==0.2.5
!pip install librosa==0.6.0
!pip install scipy==1.0.0
!pip install Unidecode==1.0.22
!pip install pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for matplotlib: filename=matplotlib-3.2.2-cp311-cp311-linux_x86_64.whl size=11963198 sha256=d4242bb4d8396ebf6cad725c331fb406aacf1ad40528f232960e9ff89e689635
  Stored in directory: /root/.cache/p

ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.2 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==1.15.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: inflect
    Found existing installation: inflect 7.5.0
    Uninstalling inflect-7.5.0:
      Successfully uninstalled inflect-7.5.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.5 MB/s eta 0:00:00
  Created wheel for 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scipy
  Running setup.py clean for scipy
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for scipy
Failed to build scipy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (scipy)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
!apt-get install -y libsndfile1 ffmpeg
!pip install matplotlib numpy scipy librosa unidecode inflect
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Looking in indexes: https://download.pytorch.org/whl/cu118
ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [ ]:
pip install torch==1.9.0  # Replace with a compatible version


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.4/735.4 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0


In [9]:
import zipfile
import os

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/processed_data.zip'

# Directory where you want to unzip the file
unzip_dir = '/content/processed_data'

# Ensure the directory exists
os.makedirs(unzip_dir, exist_ok=True)

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f"Files extracted to {unzip_dir}")


Files extracted to /content/processed_data


In [10]:
import os
import glob

# Specify the directory (current directory)
directory = "/content/processed_data"

# Get all files except .npy files
files_to_delete = [f for f in glob.glob(directory + "/*") if not f.endswith(".npy")]

# Delete the files
for file in files_to_delete:
    try:
        os.remove(file)
        print(f"Deleted: {file}")
    except Exception as e:
        print(f"Error deleting {file}: {e}")


Deleted: /content/processed_data/metadata.json


In [ ]:
import os
import json
import random
import shutil

# Path to your JSON file
json_file = '/content/metadata.json'
output_dir = '/content/metadata'  # Replace with your desired output directory

# Paths for train, valid, and transcripts
train_dir = os.path.join(output_dir, 'train')
valid_dir = os.path.join(output_dir, 'valid')
transcripts_dir = os.path.join(output_dir, 'transcripts')

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(transcripts_dir, exist_ok=True)

# Load JSON data
with open(json_file, 'r') as f:
    data = json.load(f)

# List to hold all filenames and transcriptions
all_data = []

# Prepare transcripts and create file list
for audio_id, details in data.items():
    mel_filename = f'{audio_id}.npy'
    transcription = details['text']

    # Save transcript to a file
    with open(os.path.join(transcripts_dir, f'{audio_id}.txt'), 'w') as f:
        f.write(transcription)

    # Add to the data list
    all_data.append(f'{mel_filename}|{transcription}')

# Shuffle the data
random.shuffle(all_data)

# Split data into 90% train and 10% valid
train_size = int(0.9 * len(all_data))
train_data = all_data[:train_size]
valid_data = all_data[train_size:]

# Save train and valid files
with open(os.path.join(train_dir, 'train.txt'), 'w') as f:
    f.write('\n'.join(train_data))

with open(os.path.join(valid_dir, 'valid.txt'), 'w') as f:
    f.write('\n'.join(valid_data))

print("✅ Train and validation files created successfully.")


✅ Train and validation files created successfully.


In [ ]:
import shutil
import os

def move_subfolder(source_folder, destination_folder):
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)  # Create destination folder if it doesn't exist

    folder_name = os.path.basename(source_folder)
    destination_path = os.path.join(destination_folder, folder_name)

    try:
        shutil.move(source_folder, destination_path)
        print(f"Moved '{source_folder}' to '{destination_path}' successfully.")
    except Exception as e:
        print(f"Error moving folder: {e}")

# Example usage
source = "/content/mels"
destination = "/content/tacotron2"

move_subfolder(source, destination)


Moved '/content/mels' to '/content/tacotron2/mels' successfully.


In [ ]:
import librosa
import numpy as np

# Path to any original audio file from your dataset
audio_path = "/content/LJ001-0005.wav"

# Load the audio
y, sr = librosa.load(audio_path, sr=None)

print(f"Detected Sampling Rate: {sr}")


Detected Sampling Rate: 22050


In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Character Embedding Layer
class CharacterEmbedding(nn.Module):
    def __init__(self, num_chars, embedding_dim):
        super(CharacterEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_chars, embedding_dim)

    def forward(self, x):
        return self.embedding(x)

# Encoder: Convolutions + Bidirectional LSTM
class Encoder(nn.Module):
    def __init__(self, embedding_dim, conv_channels, lstm_hidden):
        super(Encoder, self).__init__()
        self.convolutions = nn.Sequential(
            nn.Conv1d(embedding_dim, conv_channels, kernel_size=5, padding=2),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.Conv1d(conv_channels, conv_channels, kernel_size=5, padding=2),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.Conv1d(conv_channels, conv_channels, kernel_size=5, padding=2),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU()
        )
        self.lstm = nn.LSTM(conv_channels, lstm_hidden, batch_first=True, bidirectional=True)

    def forward(self, x):
        x = x.transpose(1, 2)  # Change shape for Conv1d (B, T, C) → (B, C, T)
        x = self.convolutions(x)
        x = x.transpose(1, 2)  # Restore (B, T, C)
        x, _ = self.lstm(x)
        return x

# Prenet: Regularize Decoder Input
class Prenet(nn.Module):
    def __init__(self, in_dim, sizes=[256, 128]):
        super(Prenet, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(in_dim, size) for size in sizes])

    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
            x = F.dropout(x, p=0.5, training=self.training)
        return x

# Multi-Head Attention (Improved Attention Layer)
class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert hidden_dim % num_heads == 0
        self.num_heads = num_heads
        self.dim_per_head = hidden_dim // num_heads

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, key, value):
        B, T, C = query.size()

        # Linear projections for query, key, value
        query = self.query(query).view(B, T, self.num_heads, self.dim_per_head).transpose(1, 2)
        key = self.key(key).view(B, T, self.num_heads, self.dim_per_head).transpose(1, 2)
        value = self.value(value).view(B, T, self.num_heads, self.dim_per_head).transpose(1, 2)

        # Scaled Dot-Product Attention
        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.dim_per_head ** 0.5)
        attn = F.softmax(scores, dim=-1)

        # Context Vector
        context = torch.matmul(attn, value).transpose(1, 2).contiguous().view(B, T, C)

        return self.out(context)

# Decoder: LSTM + Attention + Stop Token
class Decoder(nn.Module):
    def __init__(self, mel_dim, hidden_dim, num_heads):
        super(Decoder, self).__init__()

        # Assuming a basic decoder structure with a linear layer
        self.attention = MultiHeadAttention(hidden_dim, num_heads)  # Replace with your attention mechanism
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)  # Assuming LSTM for the decoder
        self.linear = nn.Linear(hidden_dim, mel_dim)  # Convert to mel spectrogram output size

    def forward(self, encoder_outputs, mel_input):
        # Apply attention and LSTM layers
        attention_output = self.attention(encoder_outputs, encoder_outputs, encoder_outputs)  # Simplified attention mechanism
        rnn_output, _ = self.rnn(attention_output)

        # Pass through the linear layer to get mel spectrogram
        mel_output = self.linear(rnn_output)
        stop_token = mel_output[:, -1]  # Assuming stop token is the last frame

        return mel_output, stop_token


# Postnet: Mel-Spectrogram Refinement
class Postnet(nn.Module):
    def __init__(self, mel_dim, postnet_dim):
        super(Postnet, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(mel_dim, postnet_dim, kernel_size=5, padding=2),
            nn.BatchNorm1d(postnet_dim),
            nn.Tanh(),
            nn.Conv1d(postnet_dim, mel_dim, kernel_size=5, padding=2)
        )

    def forward(self, x):
        return self.layers(x.transpose(1, 2)).transpose(1, 2)

# Full Tacotron 2 Model
class Tacotron2(nn.Module):
    def __init__(self, num_chars, embedding_dim, mel_dim, hidden_dim, num_heads, postnet_dim):
        super(Tacotron2, self).__init__()
        self.embedding = CharacterEmbedding(num_chars, embedding_dim)
        self.encoder = Encoder(embedding_dim, hidden_dim, hidden_dim)
        self.decoder = Decoder(mel_dim, hidden_dim, num_heads)
        self.postnet = Postnet(mel_dim, postnet_dim)

    def forward(self, text, mel_input):
        # Embedding + Encoding
        embedded_text = self.embedding(text)
        encoder_outputs = self.encoder(embedded_text)

        # Decode to mel-spectrogram + stop token
        mel_output, stop_token = self.decoder(encoder_outputs, mel_input)

        # Postnet refinement with residual learning
        mel_output = mel_output + self.postnet(mel_output)

        return mel_output, stop_token


# Updated Training Loop with Padding / Cropping for Mismatch Handling
def train_model(model, train_loader, optimizer, criterion_mel, criterion_stop, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (text, mel, mel_shape) in enumerate(train_loader):
            optimizer.zero_grad()

            # Forward pass
            mel_output, stop_token = model(text, mel)

            # Handle target mel spectrogram size mismatch by padding/cropping
            mel_length = mel_output.size(1)
            target_length = mel.size(1)

            # Padding or cropping the target mel spectrogram to match predicted mel sequence length
            if target_length < mel_length:
                mel = mel[:, :mel_length, :]  # Crop the target mel
            elif target_length > mel_length:
                padding_size = target_length - mel_length
                mel = F.pad(mel, (0, padding_size))  # Pad the target mel

            # Compute losses
            mel_loss = criterion_mel(mel_output, mel)
            stop_token_loss = criterion_stop(stop_token.squeeze(), torch.ones_like(stop_token.squeeze()))  # assuming all tokens are '1'

            loss = mel_loss + stop_token_loss
            loss.backward()

            # Optimization step
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    return model


In [28]:
import os
import json
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm
import torch.nn.utils.rnn as rnn_utils

# Path to dataset
DATA_PATH = '/content/processed_data'  # Update this path to where your data is stored
METADATA_PATH = '/content/metadata.json'  # Path to metadata.json

# Load metadata from JSON
with open(METADATA_PATH, 'r') as f:
    metadata = json.load(f)

# Custom Dataset class
class Tacotron2Dataset(Dataset):
    def __init__(self, data_path, metadata, max_length=1500):
        self.data_path = data_path
        self.metadata = metadata
        self.max_length = max_length

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        file_id = list(self.metadata.keys())[idx]
        mel_path = os.path.join(self.data_path, f'{file_id}.npy')
        mel_spectrogram = np.load(mel_path)

        text = self.metadata[file_id]['text']
        mel_shape = self.metadata[file_id]['mel_shape']

        # Convert text to indices (here we assume text is preprocessed to indices)
        text_indices = self.text_to_sequence(text)

        if len(text_indices) > self.max_length:
            text_indices = text_indices[:self.max_length]

        return torch.tensor(text_indices), torch.tensor(mel_spectrogram), torch.tensor(mel_shape)

    def text_to_sequence(self, text):
        # Convert text to sequence of indices based on a basic char-to-index mapping
        # Extend this if necessary for more characters or a different language
        char_to_idx = {chr(i + 97): i for i in range(26)}  # Basic char-to-index for a-z
        char_to_idx.update({' ': 26, ',': 27, '.': 28})  # Add some punctuation as examples
        text_indices = [char_to_idx.get(c, 0) for c in text.lower()]  # Default to 0 for unknown chars
        return text_indices

# Collate function to pad the sequences to the max length in each batch
def collate_fn(batch):
    # Sort by sequence length in descending order
    batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

    # Separate the inputs, mel spectrograms, and mel shapes
    text_sequences, mel_spectrograms, mel_shapes = zip(*batch)

    # Pad the text sequences
    text_sequences_padded = rnn_utils.pad_sequence(text_sequences, batch_first=True, padding_value=0)

    # Pad mel spectrograms to the maximum length in the batch
    mel_lengths = [mel.shape[1] for mel in mel_spectrograms]  # Get the time dimension size
    max_mel_length = max(mel_lengths)  # Find the max length for padding

    padded_mel_spectrograms = []
    for mel in mel_spectrograms:
        padding_size = max_mel_length - mel.shape[1]
        padded_mel = np.pad(mel, ((0, 0), (0, padding_size)), mode='constant')
        padded_mel_spectrograms.append(torch.tensor(padded_mel))

    # Stack mel spectrograms and mel shapes into batches
    mel_spectrograms = torch.stack(padded_mel_spectrograms, dim=0)
    mel_shapes = torch.stack(mel_shapes, dim=0)

    return text_sequences_padded, mel_spectrograms, mel_shapes


# Initialize Dataset and DataLoader with the custom collate function
train_dataset = Tacotron2Dataset(DATA_PATH, metadata)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2, collate_fn=collate_fn)

# Model definition as in your code
class Tacotron2(nn.Module):
    def __init__(self, num_chars=30, embedding_dim=256, mel_dim=80, hidden_dim=1024, num_heads=8, postnet_dim=512):
        super(Tacotron2, self).__init__()
        self.embedding = nn.Embedding(num_chars, embedding_dim)
        self.encoder = Encoder(embedding_dim, hidden_dim, hidden_dim)

        # Add a linear layer to match encoder output (2048) to decoder input size
        self.encoder_projection = nn.Linear(2048, hidden_dim)  # Reduce from 2048 to hidden_dim (1024)

        self.decoder = Decoder(mel_dim, hidden_dim, num_heads)
        self.postnet = Postnet(mel_dim, postnet_dim)

    def forward(self, text, mel_input):
        # Embed the text
        embedded_text = self.embedding(text)
        print(f"Embedded Text Shape: {embedded_text.shape}")  # Print shape

        encoder_outputs = self.encoder(embedded_text)
        print(f"Encoder Output Shape: {encoder_outputs.shape}")  # Print shape

        # Project encoder output to decoder input size (hidden_dim)
        encoder_outputs = self.encoder_projection(encoder_outputs)
        print(f"Projected Encoder Output Shape: {encoder_outputs.shape}")  # Print shape

        # Pass encoder outputs directly to decoder without flattening
        mel_output, stop_token = self.decoder(encoder_outputs, mel_input)
        print(f"Mel Output Shape: {mel_output.shape}, Stop Token Shape: {stop_token.shape}")  # Print shape

        mel_output = mel_output + self.postnet(mel_output)
        return mel_output, stop_token


# Initialize model, optimizer, and loss functions
model = Tacotron2()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Loss function: MSE for mel spectrograms, binary cross-entropy for stop token
criterion_mel = nn.MSELoss(reduction='none')  # No reduction to handle padding manually
criterion_stop = nn.BCEWithLogitsLoss()

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (text, mel, mel_shape) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()

        # Forward pass
        mel_output, stop_token = model(text, mel)

        # Crop or pad the mel spectrogram to match predicted length
        mel_lengths = [mel.size(1) for mel in mel]
        mel_output = mel_output[:, :max(mel_lengths), :]  # Crop if output length is greater

        # Compute losses
        mel_loss = criterion_mel(mel_output, mel)  # Loss will be computed element-wise
        stop_token_loss = criterion_stop(stop_token.squeeze(), torch.ones_like(stop_token.squeeze()))  # assuming all tokens are '1'

        # Mask out the loss for padded sections
        mel_loss = mel_loss.sum() / mel_loss.size(0)  # Averaging across batch size

        loss = mel_loss + stop_token_loss
        loss.backward()

        # Optimization step
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

    # Save model checkpoint every few epochs
    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f"/content/tacotron2_epoch_{epoch+1}.pth")


IndentationError: expected an indented block after function definition on line 29 (<ipython-input-28-f2166496de5b>, line 30)